In [1]:
%matplotlib qt

In [2]:
## Dependencies
from libs.timing_diagram import *
from libs.design_functions import *
import numpy as np
import matplotlib.pyplot as plt
from libs.spherical_earth_geometry_radar import *
from libs.radartools.farField import UniformAperture
from libs.ambiguity_functions import *
from unambiguous_imaging_design_analysis import *
import matplotlib as mpl

ChatGpt: Sure! Here's the specification for loading the analysis results from the saved pickle files:

### LOADER
- Load the analysis results from the saved pickle files using pickle module.
- Load the following individual files from the "analysis_results" folder:
    * results_list.pk
    * x_coordinate.pk
    * y_coordinate.pk
    * undersampling_list.pk
    * rasr_free_swath_list.pk
    * core_snr_list.pk
- Assign the loaded data to the corresponding variable names:
    * results_list
    * x_coordinate
    * y_coordinate
    * undersampling_list
    * rasr_free_swath_list
    * core_snr_list

Note: The file names and variable names must match with the ones used in the SAVER section.
Note1: Run the cells preceding and in Phase 0 before or after loading

In [3]:
import pickle
from IPython.display import Javascript

Javascript("Jupyter.notebook.execute_cell_range(1,11)")  #doesn't work. run clls manually up to phase0

# Load the results_list
with open('analysis_results/results_list.pk', 'rb') as f:
    results_list = pickle.load(f)
# Load the x_coordinate
with open('analysis_results/x_coordinate.pk', 'rb') as f:
    x_coordinate = pickle.load(f)
# Load the y_coordinate
with open('analysis_results/y_coordinate.pk', 'rb') as f:
    y_coordinate = pickle.load(f)
# Load the undersampling_list
with open('analysis_results/undersampling_list.pk', 'rb') as f:
    undersampling_list = pickle.load(f)
# Load the rasr_free_swath_list
with open('analysis_results/rasr_free_swath_list.pk', 'rb') as f:
    rasr_free_swath_list = pickle.load(f)
# Load the core_snr_list
with open('analysis_results/core_snr_list.pk', 'rb') as f:
    core_snr_list = pickle.load(f)

In [4]:
import json

with open("analysis_results/settings.json", "r") as read_file:
    data = json.load(read_file)

La = data["antenna_l"]
Wa = data["antenna_w"]
wavelength = data["wavelength"]
vs = data["orbital_speed"]
h = data["altitude"]
dutycycle = data["dutycycle"]
nadir_duration = data["nadir_duration"]

# Selection
filter by swath width
and
Azimuth resolution

In [5]:
# todo define these as arrays then make a meshgrid for all the combinations
minimum_ground_swath = np.array([20e3])  # m
minimum_azimuth_resolution = np.array([5])  # m

# MESHGRIDS i.e. all combinations
GS, AR = np.meshgrid(minimum_ground_swath, minimum_azimuth_resolution)

# flatten them all
GS = GS.flatten()
AR = AR.flatten()

In [6]:
# unpack the corrected azimuth resolution and swath widths
azimuth_resolutions = []
swathes = []
for dictionary in results_list:
    ar = float(dictionary["corrected-azimuth-resolution-underprocessed"])
    sw = float(dictionary["usable-rasr-swath"])
    azimuth_resolutions.append(ar)
    swathes.append(sw)

In [7]:
# convert dictionaries to numpy arrays
azimuth_resolutions = np.array(azimuth_resolutions)
swathes = np.array(swathes)

In [8]:
# thresholded lists
filtered_designs = []  # list of index vectors

for ii in range(len(AR)):
    indexes1 = np.argwhere(azimuth_resolutions <= AR[ii])
    indexes2 = np.argwhere(swathes >= GS[ii])
    intersection_set = np.intersect1d(indexes1, indexes2)
    filtered_designs.append(intersection_set)

# Design filter function
formalization of the above
it shall get as the inputs:
a np array of minimum ground swathes
a np array  of minimum azimuth resolution
a list of result dictionaries
as output:
a list of numpy array containing the indices corresponding to the chosen dictionaries for every minimum ground swath -- minimum azimuth resolution pairs
the reference GS AR pairs as list

In [9]:
#todo
def designs_filter(min_swath, min_az_resolution, results):
    """

    :param min_swath: numpy array of minimum swathes to select
    :param min_az_resolution: numpy array of minimum azimuth resolutions to select
    :param results:
    :return:    filtered_designs: list of numpy arrays containing the indexes of the selected designs for a min_swat - min_az_resolution pair
                boundaries: list of minimum swath and azimuth resolution corresponding to the filtered_designs list elements.
    """
    # MESHGRIDS i.e. all combinations
    GS, AR = np.meshgrid(min_swath, min_az_resolution)
    # flatten them all
    GS = GS.flatten()
    AR = AR.flatten()
    # unpack the corrected azimuth resolution and swath widths
    azimuth_resolutions = []
    swathes = []
    for dictionary in results:
        ar = float(dictionary["corrected-azimuth-resolution-underprocessed"])
        sw = float(dictionary["usable-rasr-swath"])
        azimuth_resolutions.append(ar)
        swathes.append(sw)
    # convert dictionaries to numpy arrays
    azimuth_resolutions = np.array(azimuth_resolutions)
    swathes = np.array(swathes)
    # thresholded lists
    filtered_designs = []  # list of index vectors
    # reference parameters
    boundaries = []
    for ii in range(len(AR)):
        indexes1 = np.argwhere(azimuth_resolutions <= AR[ii])
        indexes2 = np.argwhere(swathes >= GS[ii])
        intersection_set = np.intersect1d(indexes1, indexes2)
        filtered_designs.append(intersection_set)
        boundaries.append({'minmum-azimuth-resolution': AR[ii], 'minimum-ground-swath': GS[ii]})

    return filtered_designs, boundaries

In [10]:
#test
filtered_designs, boun = designs_filter(minimum_ground_swath, minimum_azimuth_resolution, results_list)

# Winning sets visualization on TD diagram
plot a time diagram and display the location of the winning sets

In [11]:
## canonical constraints line plots
# 1 ground range axis in m
ground_range = np.linspace(0, 2000, 500) * 1000

prf_opt = optimal_prf_line(ground_range, Wa, wavelength, h)

fig, (ax, ax1) = plt.subplots(2, dpi=150, gridspec_kw={'height_ratios': [30, 1]})

# minimum lines
prf_min = minimum_prf_line(ground_range, vs, La, h)
PRI = 1 / 7050
prf = np.linspace(1 / PRI - 1500, 1 / PRI + 10000, 100)

time_diagram_plotter(ax, prf, dutycycle, h, nadir=False, integrationtime=False, color='dimgray')
nadir_return_plotter(ax, prf, dutycycle, nadir_duration, h, color='silver')
ax.set_xlabel('PRF [Hz]')
ax.set_ylabel(' Ground range [km]')
ax.set_xlim(1 / PRI - 1000, 1 / PRI + 1000)
ax.set_ylim(100, 300)
## canonical constraints line plots
ax.plot(prf_opt, ground_range / 1000, 'k')
ax.plot(prf_min, ground_range / 1000, '--k')
ax.set_title('Winners visualization')
fig.tight_layout()

C:\Users\smen851\PycharmProjects\systemtradeoffs\unambiguous_imaging_design_analysis.py:55: RuntimeWarning: divide by zero encountered in divide
  prf_opt = 1 / pri
C:\Users\smen851\PycharmProjects\systemtradeoffs\libs\design_functions.py:42: RuntimeWarning: invalid value encountered in arccos
  beta = arccos(((re + h) ** 2 - re ** 2 + rs ** 2) / (2 * (re + h) * rs))
C:\Users\smen851\PycharmProjects\systemtradeoffs\libs\design_functions.py:43: RuntimeWarning: invalid value encountered in arccos
  alpha = arccos(((re + h) ** 2 + re ** 2 - rs ** 2) / (2 * (re + h) * re))


In [12]:
# plotting the winner samples
for ii in filtered_designs[0]:
    ax.plot(x_coordinate[ii], y_coordinate[ii] / 1000, 'x', color='black')
# adjust the plot scale
ax.set_xlim(np.array(x_coordinate).min() - 3e2, np.array(x_coordinate).max() + 3e2)
ax.set_ylim(np.array(y_coordinate).min() / 1e3 - 30, np.array(y_coordinate).max() / 1e3 + 30);

2. make a underprocessing - swath plot

In [13]:
filtered_results_list = [results_list[i] for i in filtered_designs[0]]
len(filtered_results_list)


115

In [14]:
# 1 create another timing diagram in black and white:
# plot
fig, (ax, ax1) = plt.subplots(2, dpi=150, gridspec_kw={'height_ratios': [30, 1]})
time_diagram_plotter(ax, prf, dutycycle, h, nadir=False, integrationtime=False, color='dimgray')
nadir_return_plotter(ax, prf, dutycycle, nadir_duration, h, color='silver')
ax.set_xlabel('PRF [Hz]')
ax.set_ylabel(' Ground range [km]')
ax.set_xlim(1 / PRI - 1000, 1 / PRI + 1000)
ax.set_ylim(100, 300)
## canonical constraints line plots
ax.plot(prf_opt, ground_range / 1000, 'k')
ax.plot(prf_min, ground_range / 1000, '--k')
ax.set_title('Doppler under-processing ratio visualization')
fig.tight_layout()

# adjust the plot scale
ax.set_xlim(min(x_coordinate) - 3e2, max(x_coordinate) + 3e2)
ax.set_ylim(min(y_coordinate) / 1e3 - 50, max(y_coordinate) / 1e3 + 50)
# colored lines:
for dictionary in filtered_results_list:
    design_point_visualizer(ax, dictionary, 'minimum-undersampling', 'plasma', 0, 1,
                            logscale=False, h=h, cmapaxis=ax1, labeling=False)
for ii in filtered_designs[0]:
    ax.plot(x_coordinate[ii], y_coordinate[ii] / 1000, 'x', color='black', zorder=15)

fig.tight_layout()

# Boundaries
draw a curve on the T-D plot corresponding to the edges of the swathes within selection

1. Assuming the validity region shape to be convex, find the samples lying onto the edges
2. use a boxcar filter to include more prf samples
3. store the max and min ranges in 2 max min vectors (this will be the 2 edges lines)
4. the same thing could be done using the swath extremes

In [15]:
for ii in filtered_designs[0]:
    print(x_coordinate[ii])


6114.192364675287
6169.468746584834
6224.745128494382
6224.745128494382
6280.02151040393
6335.297892313477
6390.574274223025
6445.850656132573
6501.12703804212
6556.403419951668
6556.403419951668
6611.679801861216
6666.956183770763
6722.232565680311
6777.508947589859
6832.7853294994065
6888.061711408955
6888.061711408955
6943.338093318503
6998.61447522805
7053.890857137598
7109.167239047146
7164.443620956694
7219.720002866241
7274.996384775789
7330.272766685337
7385.549148594884
7440.825530504432
7496.10191241398
7551.378294323527
7606.654676233075
7661.931058142623
7717.207440052171
7772.483821961719
7827.760203871267
7883.036585780814
7938.312967690362
7993.58934959991
8048.865731509457
8159.418495328553
8214.6948772381
8269.97125914765
8325.247641057196
8380.524022966743
8435.800404876292
8491.07678678584
8546.353168695387
8601.629550604936
8656.905932514483
8767.458696333579
8822.735078243126
8878.011460152673
8933.287842062222
8988.564223971769
9099.116987790865
9154.393369700412


Problem the prf axis is not uniform, but is it a problem? maybe no

In [16]:
# boxcar size
box = 30  # samples
max_indexes = []
min_indexes = []
print("hellllo")
for ii in range(len(filtered_designs[0]) + box):
    if ii >= len(filtered_designs[0]):
        indexes = filtered_designs[0][ii - box: len(filtered_designs[0])]
    elif ii - box >= 0:
        indexes = filtered_designs[0][ii - box: ii]
    else:
        indexes = filtered_designs[0][0: ii]
    subset_range = np.array([y_coordinate[jj] for jj in indexes])
    if subset_range.size != 0:
        maxi = np.argmax(subset_range)
        mini = np.argmin(subset_range)
        if indexes[maxi] not in max_indexes:  # if not in list already
            max_indexes.append(indexes[maxi])
        if indexes[mini] not in min_indexes:
            min_indexes.append(indexes[mini])

hellllo


In [17]:
filtered_designs[0][-5:2]

array([], dtype=int64)

In [18]:
# plot the lines
x_coordinate = np.array(x_coordinate)
y_coordinate = np.array(y_coordinate)
ax.plot(x_coordinate[min_indexes], y_coordinate[min_indexes] * 1e-3, 'r')
ax.plot(x_coordinate[max_indexes], y_coordinate[max_indexes] * 1e-3, 'g')

# Boundary visualization function
does the same thing of above, but takes the extremes of the usefull swath and plots the broadside pont of the edge points
accepts as argument a string to format the line

In [19]:
# prototype
def design_set_boundary(axis, results, filtered_indexes, box=30, linestyle='k'):
    """

    :param axis: axis to plot on
    :param results: resuts list unfiltered
    :param filtered_indexes: indexes of the winning set
    :param box: boxcar filter length in sample default 30
    :param linestyle: arg variable for plot configuration
    :return:
    """
    # retrieve the coordinates
    x_coordinate = []
    y_coordinate = []
    y_top = []
    y_bottom = []
    for ii in range(len(results)):
        dictionary = results[ii]
        x_coordinate.append(np.average(dictionary['prf']))
        y_coordinate.append(
            np.average(dictionary['compressed-ground-swath']))
        if len(dictionary['usable-swath-ranges']) > 0:
            y_top.append(dictionary['usable-swath-ranges'].max())
            y_bottom.append(dictionary['usable-swath-ranges'].min())
        else:
            y_top.append(np.average(dictionary['compressed-ground-swath']))
            y_bottom.append(np.average(dictionary['compressed-ground-swath']))

    # compute the lines
    max_indexes = []
    min_indexes = []
    for ii in range(len(filtered_indexes) + box):
        indexes = 0
        if ii >= len(filtered_indexes):
            indexes = filtered_indexes[ii - box: len(filtered_indexes)]
        elif ii - box >= 0:
            indexes = filtered_indexes[ii - box: ii]
        else:
            indexes = filtered_indexes[0: ii]
        subset_range = np.array([y_coordinate[jj] for jj in indexes])
        if subset_range.size != 0:
            maxi = np.argmax(subset_range)
            mini = np.argmin(subset_range)
            if indexes[maxi] not in max_indexes:  # if not in list already
                max_indexes.append(indexes[maxi])
            if indexes[mini] not in min_indexes:
                min_indexes.append(indexes[mini])

    # plot the lines
    x_coordinate = np.array(x_coordinate)
    y_top = np.array(y_top)
    y_bottom = np.array(y_bottom)
    axis.plot(x_coordinate[min_indexes], y_bottom[min_indexes] * 1e-3, linestyle)
    axis.plot(x_coordinate[max_indexes], y_top[max_indexes] * 1e-3, linestyle)
    # close the lines
    axis.plot([x_coordinate[max_indexes[0]], x_coordinate[max_indexes[0]]],
              [y_top[max_indexes[0]] * 1e-3, y_bottom[min_indexes[0]] * 1e-3], linestyle)
    axis.plot([x_coordinate[min_indexes[-1]], x_coordinate[min_indexes[-1]]],
          [y_bottom[min_indexes[-1]] * 1e-3, y_top[max_indexes[-1]] * 1e-3], linestyle)

In [20]:
#test
design_set_boundary(ax, results_list, filtered_designs[0], linestyle='blue')